## BoW y TF-IDF - Clasificación

In [3]:
# Librerias
import json
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression

In [4]:
# Datos de negocios
negocios = open('yelp_academic_dataset_business.json',encoding="utf8")
negocios_df = pd.DataFrame([json.loads(x) for x in negocios.readlines()])
negocios.close()

In [12]:
negocios_df.head(-10)

,business_id,full_address,open,categories,city,review_count,name,neighborhoods,longitude,state,stars,latitude,type
0,rncjoVoEFUJGCUoC1JgnUA,"8466 W Peoria Ave\nSte 6\nPeoria, AZ 85345",True,"[Accountants, Professional Services, Tax Servi...",Peoria,3,Peoria Income Tax Service,[],-112.241596,AZ,5.0,33.581867,business
1,0FNFSzCFP_rGUoJx8W7tJg,"2149 W Wood Dr\nPhoenix, AZ 85029",True,"[Sporting Goods, Bikes, Shopping]",Phoenix,5,Bike Doctor,[],-112.105933,AZ,5.0,33.604054,business
2,3f_lyB6vFK48ukH6ScvLHg,"1134 N Central Ave\nPhoenix, AZ 85004",True,[],Phoenix,4,Valley Permaculture Alliance,[],-112.073933,AZ,5.0,33.460526,business
3,usAsSV36QmUej8--yvN-dg,"845 W Southern Ave\nPhoenix, AZ 85041",True,"[Food, Grocery]",Phoenix,5,Food City,[],-112.085377,AZ,3.5,33.392210,business
4,PzOqRohWw7F7YEPBz6AubA,"6520 W Happy Valley Rd\nSte 101\nGlendale Az, ...",True,"[Food, Bagels, Delis, Restaurants]",Glendale Az,14,Hot Bagels & Deli,[],-112.200264,AZ,3.5,33.712797,business
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11522,cZQAS9sdTBaME7zh49cyDQ,"1275 W Elliot Rd\nTempe, AZ 85284",True,"[Shopping, Office Equipment]",Tempe,4,Staples,[],-111.958297,AZ,3.5,33.346843,business
11523,7u4W2qVaCc7HPfSpzuQ6pQ,"18206 E Williams Field Rd\nHigley, AZ 85236",True,[],Higley,4,Aden's Hay Oasis,[],-111.690865,AZ,4.5,33.307458,business
11524,7i5phUp95guwx68g0MR-OQ,"1017 N Dobson Rd Ste 107\nMesa, AZ 85201",True,"[Barbers, Beauty & Spas]",Mesa,4,Gentleman Joes Barber Shop,[],-111.871831,AZ,5.0,33.432869,business
11525,l_HptXEKW5QdBNzgXtahRQ,"5000 S Arizona Mills Cir\nTempe, AZ 85282",True,"[Shopping, Home & Garden, Appliances]",Tempe,3,Sears Appliance Outlet,[],-111.964485,AZ,1.5,33.383123,business


In [8]:
# Datos de comentarios
comentarios = open('yelp_academic_dataset_review.json',encoding="utf8")
comentarios_df = pd.DataFrame([json.loads(x) for x in comentarios.readlines()])
comentarios.close()

In [9]:
comentarios_df.shape

(229907, 8)

In [10]:
comentarios_df.head(10)

,votes,user_id,review_id,stars,date,text,type,business_id
0,"{'funny': 0, 'useful': 5, 'cool': 2}",rLtl8ZkDX5vH5nAx9C3q5Q,fWKvX83p0-ka4JS3dc6E5A,5,2011-01-26,My wife took me here on my birthday for breakf...,review,9yKzy9PApeiPPOUJEtnvkg
1,"{'funny': 0, 'useful': 0, 'cool': 0}",0a2KyEL0d3Yb1V6aivbIuQ,IjZ33sJrzXqU-0X6U8NwyA,5,2011-07-27,I have no idea why some people give bad review...,review,ZRJwVLyzEJq1VAihDhYiow
2,"{'funny': 0, 'useful': 1, 'cool': 0}",0hT2KtfLiobPvh6cDC8JQg,IESLBzqUCLdSzSqm0eCSxQ,4,2012-06-14,love the gyro plate. Rice is so good and I als...,review,6oRAC4uyJCsJl1X0WZpVSA
3,"{'funny': 0, 'useful': 2, 'cool': 1}",uZetl9T0NcROGOyFfughhg,G-WvGaISbqqaMHlNnByodA,5,2010-05-27,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,_1QQZuf4zZOyFCvXc0o6Vg
4,"{'funny': 0, 'useful': 0, 'cool': 0}",vYmM4KTsC8ZfQBg-j5MWkw,1uJFq2r5QfJG_6ExMRCaGw,5,2012-01-05,General Manager Scott Petello is a good egg!!!...,review,6ozycU1RpktNG2-1BroVtw
5,"{'funny': 1, 'useful': 3, 'cool': 4}",sqYN3lNgvPbPCTRsMFu27g,m2CKSsepBCoRYWxiRUsxAg,4,2007-12-13,"Quiessence is, simply put, beautiful. Full wi...",review,-yxfBYGB6SEqszmxJxd97A
6,"{'funny': 4, 'useful': 7, 'cool': 7}",wFweIWhv2fREZV_dYkz_1g,riFQ3vxNpP4rWLk_CSri2A,5,2010-02-12,Drop what you're doing and drive here. After I...,review,zp713qNhx8d9KCJJnrw1xA
7,"{'funny': 0, 'useful': 1, 'cool': 0}",1ieuYcKS7zeAv_U15AB13A,JL7GXJ9u4YMx7Rzs05NfiQ,4,2012-07-12,"Luckily, I didn't have to travel far to make m...",review,hW0Ne_HTHEAgGF1rAdmR-g
8,"{'funny': 0, 'useful': 0, 'cool': 0}",Vh_DlizgGhSqQh4qfZ2h6A,XtnfnYmnJYi71yIuGsXIUA,4,2012-08-17,Definitely come for Happy hour! Prices are ama...,review,wNUea3IXZWD63bbOQaOH-g
9,"{'funny': 0, 'useful': 1, 'cool': 0}",sUNkXg8-KFtCMQDV6zRzQg,jJAIXA46pU1swYyRCdfXtQ,5,2010-08-11,Nobuo shows his unique talents with everything...,review,nMHhuYan8e3cONo3PornJA


In [14]:
# Seleccionar negocios Nightlife y Restaurants
nr_df = negocios_df[negocios_df.apply(lambda x: 'Nightlife' in x['categories'] or 
                                 'Restaurants' in x['categories'], axis=1)]

# Unir con los comentarios
union_df = nr_df.merge(comentarios_df, on='business_id', how='inner')

# Ignorar el resto de las columnas
nr_df2 = union_df[['business_id','name','stars_y','text','categories']]

# Crear dos clases True - Nightlife y False Restaurants
nr_df2['Y'] = nr_df2.apply(lambda x: 'Nightlife' in x['categories'], axis=1)

# Crear datos de entrenamiento y prueba
training_data, test_data = train_test_split(nr_df2,train_size=0.8, random_state=123)

/var/folders/vb/hzrrtyn525jdg34h29y0hs400000gn/T/ipykernel_34620/1828456793.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nr_df2['Y'] = nr_df2.apply(lambda x: 'Nightlife' in x['categories'], axis=1)


In [15]:
nr_df2.shape

(166038, 6)

In [16]:
nr_df2.head(4)

,business_id,name,stars_y,text,categories,Y
0,PzOqRohWw7F7YEPBz6AubA,Hot Bagels & Deli,4,FINALLY a decent place for breakfast and lunch...,"[Food, Bagels, Delis, Restaurants]",False
1,PzOqRohWw7F7YEPBz6AubA,Hot Bagels & Deli,4,Stopped in to grab a couple coffees and a bage...,"[Food, Bagels, Delis, Restaurants]",False
2,PzOqRohWw7F7YEPBz6AubA,Hot Bagels & Deli,5,Absolutely love this place. The bagels are ma...,"[Food, Bagels, Delis, Restaurants]",False
3,PzOqRohWw7F7YEPBz6AubA,Hot Bagels & Deli,5,"You can not go wrong with this place, any bage...","[Food, Bagels, Delis, Restaurants]",False


In [17]:
training_data.shape

(132830, 6)

In [18]:
test_data.shape

(33208, 6)

In [19]:
training_data.tail(-10)

,business_id,name,stars_y,text,categories,Y
110237,JhupPnWfNlMJivnWB5druA,AZ 88,5,I recently visited AZ 88 and ventured away fro...,"[Bars, American (Traditional), Nightlife, Loun...",True
101921,hwXFp8FKJ7ivQb8_f4X23g,La's Thai Cuisine,5,"Craving takeout food, this is our go to place....","[Thai, Restaurants]",False
38937,NaXqRvoW2Zg2wSghf9JqaA,Kwan & Wok Chinese Restaurant,4,Kwan and Wok may be a fast food Chinese restau...,"[Chinese, Restaurants]",False
98602,f1ffUvX5_rV8PhAceZo5tw,Bibiano's Mexican Restaurant,4,Awesome family owned and operated Mexican rest...,"[Mexican, Restaurants]",False
98618,z0xEfgr1-lDqVatrX-rf-A,Rice To You,5,Love the food here. Very generous portion and...,"[Chinese, Restaurants]",False
...,...,...,...,...,...,...
146449,n7OJsakQko7lxGnbv1C6yg,InPockets Restaurant,5,If you're looking for an awesome breakfast pla...,"[Bakeries, Food, Sandwiches, Restaurants]",False
119906,pYaTHEgcpSei6xfkz2GUow,Chick-Fil-A,5,Lot s of Chick-fil-a stores around and all hav...,"[Chicken Wings, Restaurants]",False
17730,OmxAvtMyIvCVm16NFIOOUA,TGI Friday's,3,You know those bundles of ads you get in the m...,"[American (Traditional), Restaurants]",False
28030,mQfT3JYu18HN22DVylcE7A,Bertha's Café,3,I'm hoping that my visit to Bertha's was just ...,"[Bakeries, Food, Breakfast & Brunch, Sandwiche...",False


In [20]:
test_data.tail(-5)

,business_id,name,stars_y,text,categories,Y
132237,lvpXyfTDF5WZnjzc36r5Wg,Lucky Boy Burger Shop,4,Well after so so many years of driving by it a...,"[Burgers, Restaurants]",False
26914,-3xbryp44xhpN4BohxXDdQ,The Keg Steakhouse & Bar,3,This is a chain and I've been to the Phoenix l...,"[Bars, Steakhouses, Nightlife, Restaurants]",True
18383,cgNIBHS2mLdbTRiodAT6EQ,Tacos De Juarez,2,Maybe the current economy has forced Tacos de ...,"[Mexican, Restaurants]",False
56920,I4vC08JXAFjXszUZAszh2Q,Native New Yorker,1,Decided to give it a 2nd chance after a terrib...,"[American (New), Restaurants]",False
143245,tHztlek7Ecs-5ZonXUATyA,Apple Spice Junction,5,Imagine this: You're stuck in a 2.5 hour train...,"[Delis, Restaurants, Event Planning & Services...",False
...,...,...,...,...,...,...
53539,cdacUBBL2tDbDnB1EfhpQw,Cafe Carumba,4,A group of 7 of us came by at 7 on a Friday ni...,"[American (New), Mexican, Restaurants]",False
119190,XWvht_1ZLdK7EHJ3jo4q0g,The White Chocolate Grill,5,"This place was really, really good. So good I ...","[American (Traditional), Restaurants]",False
66655,OjNTZTZ6HYQghJN-3jmwnA,Black Angus Steakhouse,4,The molasses bread here is to die for as well ...,"[American (Traditional), Steakhouses, Seafood,...",False
117014,Dh-jKrXlivowMGDtfD_gug,The Pita Pit,5,They have 24/7 delivery! And the food tastes a...,"[Mediterranean, Sandwiches, Restaurants]",False


In [21]:
# Feature engineering
# Transformar a bag-of-words
bow_transform = CountVectorizer()
X_tr_bow = bow_transform.fit_transform(training_data['text'])
X_te_bow = bow_transform.transform(test_data['text'])
print(len(bow_transform.vocabulary_))

y_tr = training_data['Y']
y_te = test_data['Y']

# Transformar con TF-IDF
tfidf_trfm = TfidfTransformer(norm=None)
X_tr_tfidf = tfidf_trfm.fit_transform(X_tr_bow)
X_te_tfidf = tfidf_trfm.transform(X_te_bow)

84189


In [22]:
X_tr_bow.shape

(132830, 84189)

In [23]:
X_tr_tfidf

<132830x84189 sparse matrix of type '<class 'numpy.float64'>'
	with 10984605 stored elements in Compressed Sparse Row format>

In [24]:
# Clasificación
modelo1 = LogisticRegression()
modelo1.fit(X_tr_bow, y_tr)
print("Modelo con BoW")
modelo1.score(X_te_bow, y_te)

Modelo con BoW


/Users/ivan/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8888821970609492

In [25]:
modelo2 = LogisticRegression()
modelo2.fit(X_tr_tfidf, y_tr)
print("Modelo con TFIDF")
modelo2.score(X_te_tfidf, y_te)

Modelo con TFIDF


/Users/ivan/anaconda3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.8867441580342086

In [ ]:
# Tarea - Balancear clases